In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import plot_importance
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,KFold
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline

df_train = pd.read_csv('/content/drive/MyDrive/데이콘/jobcare/dacon_jobcare/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/데이콘/jobcare/dacon_jobcare/test.csv')
df_sample = pd.read_csv('/content/drive/MyDrive/데이콘/jobcare/dacon_jobcare/sample_submission.csv')
df_d_att = pd.read_csv('/content/drive/MyDrive/데이콘/jobcare/dacon_jobcare/속성_D_코드.csv')
df_h_att = pd.read_csv('/content/drive/MyDrive/데이콘/jobcare/dacon_jobcare/속성_H_코드.csv')
df_l_att = pd.read_csv('/content/drive/MyDrive/데이콘/jobcare/dacon_jobcare/속성_L_코드.csv')

In [ ]:
df_d_att

In [ ]:
'''
def feature_engine4(df, df1):
    #person_prefer_d_2, person_prefer_d_3, contents_attribute_d
    df_dict_d2_contribute = {}
    for k,v in zip(df1.iloc[:,0], df1.iloc[:,1:5]) :
        df_dict_d2_contribute[k] = v
    return df_dict_d2_contribute

a = feature_engine4(df_train, df_d_att)
a
'''
a = df_train['contents_attribute_d']
b = df_d_att.set_index('속성 D 코드').T.to_dict()
for i, v in b.items():
    if i==4 and v['속성 D 대분류코드'] ==1:
        print(i)

In [ ]:
def feature_engine3(df):
    result = []
    result1 = []
    for i in range(df.shape[0]):
        if df['person_prefer_c'].values[i]==df['contents_attribute_c'].values[i]:
            result.append(True)
        else :
            result.append(False)
        result1.append(np.abs(df['person_prefer_e'].values[i]-df['contents_attribute_e'].values[i]))
    df['condition_c'] = result
    df['condition_e'] = result1
    df.drop(['id','contents_open_dt'], axis = 1, inplace =True)
    return df
df_train = feature_engine3(df_train)
df_test = feature_engine3(df_test)




In [ ]:
'''
df_train['contents_open_dt'] = df_train['contents_open_dt'].apply(pd.to_datetime)
df_test['contents_open_dt'] = df_test['contents_open_dt'].apply(pd.to_datetime)

def feature_engine(df_train, df_test):
    list_unit = [df_train, df_test]
    for i in list_unit:
        i['year']= i['contents_open_dt'].apply(lambda x : x.year)
        i['month'] = i['contents_open_dt'].apply(lambda x : x.month)
        i['day'] = i['contents_open_dt'].apply(lambda x : x.day)
        i['hour'] = i['contents_open_dt'].apply(lambda x : x.hour)
        i.drop(['id','contents_open_dt'], axis = 1, inplace=True)
        
    return df_train, df_test
    

def feature_engine2(df_train):
    d_list = df_train[['d_l_match_yn','d_m_match_yn','d_s_match_yn']]
    h_list = df_train[['h_l_match_yn','h_m_match_yn','h_s_match_yn']]
    array_1 = np.zeros(d_list.shape[0])
    array_2 = np.zeros(h_list.shape[0])
    for i in range(array_1.shape[0]):
        array_1[i] = np.count_nonzero(d_list.values[i]==True)
        array_2[i] = np.count_nonzero(h_list.values[i]==True)
    df_train['new_feature1'] = array_1
    df_train['new_feature2'] = array_2
    df_train.drop(['d_l_match_yn','d_m_match_yn','d_s_match_yn','h_l_match_yn','h_m_match_yn','h_s_match_yn'], axis = 1, inplace = True)
    return df_train
'''

In [ ]:
'''
df_train, df_test = feature_engine(df_train, df_test)
df_train = feature_engine2(df_train)
df_test = feature_engine2(df_test)
'''
X_feature = df_train.drop('target', axis = 1, inplace = False)
y_feature = df_train['target']

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 300, max_depth=15)
xg_clf = XGBClassifier(learning_rate =0.03, reg_alpha = 3e-5, reg_lambda = 9e-2, max_depth = 15,
                         n_estimators = 300)
ada_clf = AdaBoostClassifier(n_estimators = 200, learning_rate = 0.03)
dt_lgbm = LGBMClassifier(learning_rate =0.03, reg_alpha = 3e-5, reg_lambda = 9e-2, max_depth = 15,
                         n_estimators = 300)

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)
    
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)): 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n.loc[train_index] 
        y_tr = y_train_n.loc[train_index] 
        X_te = X_train_n.loc[valid_index]  
        model.fit(X_tr , y_tr)
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    return train_fold_pred , test_pred_mean

rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_feature, y_feature, df_test, 5)
xg_train, xg_test = get_stacking_base_datasets(xg_clf, X_feature, y_feature, df_test,  5)    
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_feature, y_feature, df_test, 5)

Stack_final_X_train = np.concatenate((rf_train, xg_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((rf_test, xg_test, ada_test), axis=1)

dt_lgbm.fit(Stack_final_X_train,  y_feature)
pred = dt_lgbm.predict(Stack_final_X_test)
pred_proba = dt_lgbm.predict_proba(Stack_final_X_test)[:, 1]
df_sample['target'] = pred
df_sample.to_csv('/content/drive/MyDrive/데이콘/jobcare/dacon_jobcare/submission.csv',index = False)

In [ ]:
get_clf_eval(y_feature, pred, pred_proba)

In [ ]:
pred.shape

In [ ]:
df_sample['target'] = pred
df_sample.to_csv('/content/drive/MyDrive/데이콘/jobcare/dacon_jobcare/submission.csv',index = False)

In [ ]:
df_train

In [ ]:
a

In [ ]:
np.count_nonzero(d_list ==True)

In [ ]:
df_train[['d_l_match_yn','d_m_match_yn','d_s_match_yn']]+df_train[['h_l_match_yn','h_m_match_yn','h_s_match_yn']]